In [7]:
import pybullet as p
import time
import numpy as np
import pybullet_data 
import matplotlib.pyplot as plt
from collections import Counter
import random



pybullet build time: Dec 17 2023 23:51:54


In [6]:
class Pickup_Bot_Env():
    def __init__(self,robot,GUI=False):
        #init the bot and the environment
        if GUI:
            physicsClient = p.connect(p.GUI)
        else:
            physicsClient = p.connect(p.DIRECT)
        p.setAdditionalSearchPath(pybullet_data.getDataPath())
        planeId = p.loadURDF("plane.urdf")
        p.setGravity(0,0,-10)
        startPos1 = [-0.19670987601116390886, -0.99812458682335825078, 0.63779767016]
        # startPos1 = [0, 0, 0.]
        # startPos1 = [+0.8309593293931667, -0.5869020864131286, +0.6377976701602711]
        startOrientation1 = p.getQuaternionFromEuler([0,0,0])
        # startOrientation1 = [0, 0, 1, 0]S
        print(1)
        offset = 0.275
        ball = p.loadURDF("soccerball.urdf",[offset,0,0], globalScaling=1*0.1)
                
        self.robot = p.loadURDF("./bot/robot.urdf",startPos1, startOrientation1,useFixedBase=1)

        #setting the initial values of the 

        self.stand_rotation = 0.
        self.slider_pos = 0.32659999999973116

        self.target = []
        
    def get_reward(self):
        #get the current state and the corresponding reward function 
        #basically after the action is take you need to get the reward by assesing which state is the bot at. get the reward and send it to the main algorithm which uses it to asses the Q value 
        # this class will only be used for TD and MC. DP we need to think differently, might be slightly tricky cuz we need to mention the current policy and shit
        get_current_state()
        
        ...
    def get_current_state(self):
        #this method will be used to get the values of the current dynamics of the link which can then be used in both get_reward and in step function
        self.stand_rotation = p.getEulerFromQuaternion(p.getLinkState(self.robot , 0)[1])[2] #get the euler z rotation
        self.slider_pos = p.getLinkState(self.robot , 1)[0][2 ] #get the slide alonng the z axis
        self.gripper = p.getEulerFromQuaternion(p.getLinkState(self.robot , 2)[1])[1]
    def step(self,action):
        #take an action in pybullet
        #we can take in totol 6 actions
        ## open the gripper (1 action)
        ## close the gripper (1 action)
        ## move the slider up by 0.21 up or down (2 actions)
        ## rotate the stand by pi/2 to the left or right (2 actions)

        ##one major issue with these simulation is that they dont het you to the exact position i.e they will almost reach there but not exactly so we need to account for that face as well.
        ###one way to solve is to store the error in the state and stuff and aff that error for each time that same joint is moved

        #we have 24 states in total
        #here our objective is to move and get the reward that means we need to take the action that is speciofied by the function and then use pybullet to step then we let get reward do its thing
        get_current_state()
        
        if action=='move_left':
            p.setJointMotorControlArray(self.robot, [0],p.POSITION_CONTROL, targetPositions= [self.stand_rotation-1.57079632679])
        elif action=='move_right':
            p.setJointMotorControlArray(self.robot, [0],p.POSITION_CONTROL, targetPositions= [self.stand_rotation + 1.57079632679])
        elif action=='move_up':
            p.setJointMotorControlArray(self.robot, [1],p.POSITION_CONTROL, targetPositions= [self.slider_pos-0.21])

        elif action=='move_down':
            p.setJointMotorControlArray(self.robot, [1],p.POSITION_CONTROL, targetPositions= [self.slider_pos+0.21])

        elif action=='close_gripper':
            p.setJointMotorControlArray(self.robot, [2,3],p.POSITION_CONTROL, targetPositions= [self.gripper + 0.36179938779914940783,self.gripper + 0.36179938779914940783])#even tho the limit to the gripper is 0.26 just 
                                                                                                                                               #incase to reduce the error due to friction and stuff
        elif action=='open_gripper':
            p.setJointMotorControlArray(self.robot, [2,3],p.POSITION_CONTROL, targetPositions= [self.gripper-0.62359877559829881566,self.gripper-0.62359877559829881566])
        for i in range(100):
            p.stepSimulation()
            
    def reset_env(self):
        #p.discopnenct and call the class again
        p.disconnect()
        self.__init__(robot,GUI=False)
        
      